Before running this Notebook make sure you have the following Packages:
*`snowflake-ml-python`

You also need to add the `demo_data.py`file as a **Stage Package**

In [1]:
# Import python packages
import streamlit as st

from snowflake.ml.feature_store import (
    FeatureStore,
    CreationMode)

from snowflake.ml.registry import Registry

# from snowflake.snowpark import Session
import snowflake.snowpark.functions as snow_funcs

# Python script stored on a Snowflake stage
from demo_data import generate_demo_data

# Get the Snowpark session
from snowflake.snowpark.context import get_active_session
session = get_active_session()


In [10]:
# session = Session.builder.config("connection_name", 'mstellwall_aws_us_west3').create()
st.write("Current role: " + session.get_current_role() + ", Current schema: " + session.get_fully_qualified_current_schema() + ", Current WH: " + session.get_current_warehouse())

Current role: "SYSADMIN", Current schema: "DEMO_DB"."PUBLIC", Current WH: "COMPUTE_WH"


In [3]:
db_name = "SNOWPARK_DEMO_DB"
schema_name = "SIMPLE_ML_SCHEMA"
fs_schema_name = "SIMPLE_FS_SCHEMA"
mr_schema_name = "SIMPLE_MR_SCHEMA"
wh_name = "SIMPLE_ML_WH"

session.use_schema(f'{db_name}.{schema_name}')
session.use_warehouse(wh_name)

## Generate new customers for inference

Generate 100 new customers to use, we will have to wait around one minute before they are part of our feature store (the schedule we setup during registring the features)

In [4]:
# Start by generating some new data that we will use for inference
session.use_schema(f'{db_name}.{schema_name}')

# Generate new customers for year 2024
generate_demo_data(session, num_customers=100, month=9, start_year=2024, end_year=2024)

Added 1000 customers to table: CUSTOMER_LIFE_TIME_VALUE
Added 1000 customers to table: CUSTOMER_GENERAL_DATA
Added 1000 customers to table: CUSTOMER_BEHAVIOR_DATA
Added 100 customers to table: CUSTOMER_LIFE_TIME_VALUE
Added 100 customers to table: CUSTOMER_GENERAL_DATA
Added 100 customers to table: CUSTOMER_BEHAVIOR_DATA
------------------------
|"EMAIL"               |
------------------------
|jq7ho1e2K7@KDDoU.com  |
|TynfegHXC5@EK8gf.com  |
|LaU10PhN7X@OCtzz.com  |
|9Wphdu46yz@kmSyh.com  |
|8ktBu2ygwj@U1PcJ.com  |
|QAUnHdGliO@vsOQl.com  |
|9Lnm15ODaM@fO9Ss.com  |
|JVCgRooPMw@sV6cz.com  |
|PZSjb31f9L@Kp66W.com  |
|PW77M5pEsO@epc5D.com  |
------------------------



Greate a Spine DataFrame with the new customers, which will be used to get the features we have for them

In [12]:
# Retrieve new customers
new_customers_df = session.table(f'{db_name}.{schema_name}.CUSTOMER_LIFE_TIME_VALUE').filter(snow_funcs.col('YEAR_MONTH')=='20249').select('EMAIL')
new_customers_df.show()

------------------------
|"EMAIL"               |
------------------------
|jq7ho1e2K7@KDDoU.com  |
|TynfegHXC5@EK8gf.com  |
|LaU10PhN7X@OCtzz.com  |
|9Wphdu46yz@kmSyh.com  |
|8ktBu2ygwj@U1PcJ.com  |
|QAUnHdGliO@vsOQl.com  |
|9Lnm15ODaM@fO9Ss.com  |
|JVCgRooPMw@sV6cz.com  |
|PZSjb31f9L@Kp66W.com  |
|PW77M5pEsO@epc5D.com  |
------------------------



Connect to the Feature Store

In [11]:
# Connect to Feature Store
fs = FeatureStore(
    session=session, 
    database=db_name, 
    name=fs_schema_name, 
    default_warehouse=wh_name,
    creation_mode=CreationMode.FAIL_IF_NOT_EXIST,
)

Get the feature views that has the features we want to use

In [13]:
cust_fv = fs.get_feature_view(name="CUSTOMER_GENERAL_DATA_FEATURES",
                                version="V1")
behavior_fv = fs.get_feature_view(name="CUSTOMER_BEHAVIOR_DATA_FEATURES",
                                version="V1")

Retrieve features using the Spine DataFrame (can take up to a minute until values appear)  
Assumption is you only have the unique-id EMAIL and need to retrieve the features to score the model

In [14]:

new_customers_features = fs.retrieve_feature_values(new_customers_df, features=[cust_fv, behavior_fv])
new_customers_features.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"EMAIL"               |"GENDER"  |"MEMBERSHIP_STATUS"  |"MEMBERSHIP_LENGTH_DAYS"  |"AVG_SESSION_LENGTH_MIN"  |"AVG_TIME_ON_APP_MIN"  |"AVG_TIME_ON_WEBSITE_MIN"  |"APP_PRIMARY"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|jq7ho1e2K7@KDDoU.com  |MALE      |BRONZE               |204                       |10.6627                   |2.6627                 |8.6627                     |0              |
|TynfegHXC5@EK8gf.com  |MALE      |DIAMOND              |193                       |NULL                      |9.9401                 |9.9401                     |0              |
|LaU10PhN7X@OCtzz.com  |FEMALE    |BASIC                |255                       |NULL            

Connect to the Model Registry

In [15]:
ml_reg = Registry(session=session, database_name=db_name, schema_name=mr_schema_name)
ml_reg.show_models()

,created_on,name,model_type,database_name,schema_name,comment,owner,default_version_name,versions,aliases
0,2024-09-12 04:25:51.307000-07:00,CUSTOMER_LTV_MODEL,USER_MODEL,SNOWPARK_DEMO_DB,SIMPLE_MR_SCHEMA,None,SYSADMIN,MY_FIRST_MODEL_VERSION,"[""MY_FIRST_MODEL_VERSION""]","{""DEFAULT"":""MY_FIRST_MODEL_VERSION"",""FIRST"":""M..."


Get a reference to the default version of the model

In [16]:
registered_model = ml_reg.get_model("CUSTOMER_LTV_MODEL").default

Check which functions we can use and the inputs/outputs for them

In [17]:
registered_model.show_functions()

[{'name': 'PREDICT',
  'target_method': 'predict',
  'target_method_function_type': 'FUNCTION',
  'signature': ModelSignature(
                      inputs=[
                          FeatureSpec(dtype=DataType.STRING, name='GENDER'),
  		FeatureSpec(dtype=DataType.STRING, name='MEMBERSHIP_STATUS'),
  		FeatureSpec(dtype=DataType.INT16, name='MEMBERSHIP_LENGTH_DAYS'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='AVG_SESSION_LENGTH_MIN'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='AVG_TIME_ON_APP_MIN'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='AVG_TIME_ON_WEBSITE_MIN'),
  		FeatureSpec(dtype=DataType.INT8, name='APP_PRIMARY')
                      ],
                      outputs=[
                          FeatureSpec(dtype=DataType.INT8, name='GENDER_OHE_FEMALE'),
  		FeatureSpec(dtype=DataType.INT8, name='GENDER_OHE_MALE'),
  		FeatureSpec(dtype=DataType.DOUBLE, name='MEMBERSHIP_STATUS_OE'),
  		FeatureSpec(dtype=DataType.INT16, name='MEMBERSHIP_LENGTH_DAYS'),
  		FeatureSpec(d

Create predictions from registered model given the retrieved features, we are saving them as a table in Snowflake

In [18]:

new_predictions = registered_model.run(new_customers_features, function_name='predict')
new_predictions.write.save_as_table('MY_PREDICTIONS', mode='overwrite')
session.table('MY_PREDICTIONS').show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"EMAIL"               |"GENDER"  |"MEMBERSHIP_STATUS"  |"AVG_SESSION_LENGTH_MIN"  |"AVG_TIME_ON_APP_MIN"  |"AVG_TIME_ON_WEBSITE_MIN"  |"GENDER_OHE_FEMALE"  |"GENDER_OHE_MALE"  |"MEMBERSHIP_STATUS_OE"  |"MEMBERSHIP_LENGTH_DAYS"  |"AVG_SESSION_LENGTH_MIN_IMP"  |"AVG_TIME_ON_APP_MIN_IMP"  |"AVG_TIME_ON_WEBSITE_MIN_IMP"  |"APP_PRIMARY"  |"LIFE_TIME_VALUE_PREDICTION"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
def my_fun():
    ...

